# Writing Efficient Code

## Introduction

Python is widely used in scientific computing for its readability and extensive ecosystem. However, pure Python can be slow for computationally intensive tasks. A simple loop that runs millions of times can take seconds in Python but milliseconds in C or Fortran.

For statistical computing, this matters. Monte Carlo simulations, bootstrap resampling, optimization algorithms, and large-scale data analysis often require executing the same operations millions of times. The difference between code that takes 10 seconds and code that takes 10 minutes can determine whether an analysis is practical.

This lecture covers techniques to make Python code faster:

* Profiling to identify bottlenecks before optimizing
* Vectorization using NumPy
* Parallelization using multiprocessing
* Just-in-time (JIT) compilation with Numba
* Interfacing with C/C++ using Cython

The key principle is: **profile first, optimize second**. Most code spends the vast majority of its time in a small fraction of the codebase. Optimizing the wrong parts wastes effort and often makes code harder to read.

## Profiling Your Code

Before optimizing, you need to know where your code spends its time. Python provides built-in tools for this: `timeit` for quick benchmarks and `cProfile` for detailed analysis.

### Using timeit

The `timeit` module measures how long a small code snippet takes to execute. It runs the code multiple times and reports the average time, which gives more reliable measurements than a single run.

In [19]:
import timeit

# Time a simple expression
time = timeit.timeit('sum(range(1000))', number=10000)
print(f"Total time for 10000 runs: {time:.4f} seconds")
print(f"Average time per run: {time/10000*1000:.4f} ms")

Total time for 10000 runs: 0.1069 seconds
Average time per run: 0.0107 ms


For timing functions, you can pass a setup string that runs once before the timing begins:

In [20]:
import timeit

setup = '''
import numpy as np
data = np.random.randn(10000)
'''

# Time NumPy operations
numpy_time = timeit.timeit('np.mean(data)', setup=setup, number=1000)
print(f"NumPy mean: {numpy_time:.4f} seconds for 1000 runs")

# Compare with pure Python
setup_python = '''
import numpy as np
data = list(np.random.randn(10000))
'''
python_time = timeit.timeit('sum(data)/len(data)', setup=setup_python, number=1000)
print(f"Python mean: {python_time:.4f} seconds for 1000 runs")
print(f"NumPy is {python_time/numpy_time:.1f}x faster")

NumPy mean: 0.0149 seconds for 1000 runs
Python mean: 0.2094 seconds for 1000 runs
NumPy is 14.0x faster


In Jupyter notebooks or IPython, you can use the `%timeit` magic command for convenience:

In [21]:
%timeit sum(range(1000))

6.83 µs ± 24.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


This automatically determines how many iterations to run and reports statistics.

### Using cProfile

While `timeit` is useful for comparing specific snippets, `cProfile` analyzes an entire program and shows where time is spent across all function calls.

In [23]:
import cProfile
import pstats
import numpy as np


def slow_variance(data):
    """Calculate variance using a slow approach."""
    n = len(data)
    mean = sum(data) / n
    total = 0
    for x in data:
        total += (x - mean) ** 2
    return total / n


def fast_variance(data):
    """Calculate variance using NumPy."""
    return np.var(data)


def main():
    data = list(np.random.randn(100000))
    data_np = np.array(data)

    for _ in range(10):
        slow_variance(data)
        fast_variance(data_np)


# Profile the main function
cProfile.run('main()', 'profile_stats')

# Print sorted results
stats = pstats.Stats('profile_stats')
stats.strip_dirs()
stats.sort_stats('cumulative')
stats.print_stats(10)

Wed Jan 28 06:56:46 2026    profile_stats

         961 function calls (951 primitive calls) in 0.135 seconds

   Ordered by: cumulative time
   List reduced from 169 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.133    0.133 <string>:1(<module>)
        1    0.004    0.004    0.132    0.132 1590960761.py:21(main)
       10    0.097    0.010    0.121    0.012 1590960761.py:6(slow_variance)
       10    0.024    0.002    0.024    0.002 {built-in method builtins.sum}
        1    0.005    0.005    0.005    0.005 {built-in method numpy.array}
        1    0.000    0.000    0.002    0.002 decorator.py:229(fun)
        1    0.000    0.000    0.002    0.002 history.py:54(only_when_enabled)
        1    0.000    0.000    0.001    0.001 history.py:826(writeout_cache)
        1    0.001    0.001    0.001    0.001 {method 'randn' of 'numpy.random.mtrand.RandomState' objects}
        1    0.000    0.000    0

The output shows:
- `ncalls`: number of times the function was called
- `tottime`: total time spent in the function (excluding subfunctions)
- `cumtime`: cumulative time including subfunctions
- `percall`: time per call

Look for functions with high `tottime` or `cumtime` values. These are your optimization targets.

### Question

Consider the following profiling output:

```
   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1000    5.234    0.005    5.234    0.005 stats.py:15(compute_distances)
     1000    0.012    0.000    0.012    0.000 stats.py:25(normalize)
   100000    0.892    0.000    0.892    0.000 stats.py:30(update_centroid)
        1    0.001    0.001    6.139    6.139 stats.py:40(kmeans)
```

Which function should you optimize first, and why?

### Answer

`compute_distances` should be optimized first. It has the highest `tottime` (5.234 seconds), meaning the program spends 85% of its total time (5.234 out of 6.139 seconds) inside this function. Although `update_centroid` is called 100 times more often, its total time is only 0.892 seconds. Optimizing `compute_distances` will have the largest impact on overall performance.

### Line Profiler

For more detailed analysis, the `line_profiler` package shows time spent on each line within a function. Install it with

In [6]:
!pip install line_profiler

In [5]:
# In a file: example.py
from line_profiler import profile


@profile
def compute_pairwise_distances(X):
    n = X.shape[0]
    distances = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            diff = X[i] - X[j]
            distances[i, j] = np.sqrt(np.sum(diff**2))
    return distances

Run with: `kernprof -l -v example.py`

Example output:

```
Timer unit: 1e-06 s

Total time: 2.45632 s
File: example.py
Function: compute_pairwise_distances at line 5

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     5                                           @profile
     6                                           def compute_pairwise_distances(X):
     7         1          2.0      2.0      0.0      n = X.shape[0]
     8         1        156.0    156.0      0.0      distances = np.zeros((n, n))
     9       101         45.0      0.4      0.0      for i in range(n):
    10     10100       3852.0      0.4      0.2          for j in range(n):
    11     10000      98234.0      9.8      4.0              diff = X[i] - X[j]
    12     10000    2354031.0    235.4     95.8              distances[i, j] = np.sqrt(np.sum(diff**2))
    13         1          0.0      0.0      0.0      return distances
```

The output shows that line 12 takes 95.8% of the total time. This tells us exactly where to focus optimization efforts: the `np.sqrt(np.sum(diff**2))` computation inside the inner loop.

## Vectorization with NumPy

The most common way to speed up numerical Python code is vectorization: replacing explicit loops with array operations. NumPy operations are implemented in C and operate on entire arrays at once, avoiding Python's interpreter overhead.

### The Cost of Python Loops

Python loops are slow because each iteration involves:
- Looking up the loop variable
- Type checking operands
- Dispatching to the appropriate operation
- Creating new Python objects for results

Consider computing the element-wise sum of two arrays:

In [24]:
import numpy as np

def sum_with_loop(a, b):
    """Element-wise sum using a Python loop."""
    n = len(a)
    result = np.empty(n)
    for i in range(n):
        result[i] = a[i] + b[i]
    return result


def sum_vectorized(a, b):
    """Element-wise sum using NumPy."""
    return a + b


# Compare performance
n = 1000000
a = np.random.randn(n)
b = np.random.randn(n)

%timeit sum_with_loop(a, b)
%timeit sum_vectorized(a, b)

114 ms ± 1.68 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
450 µs ± 9.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


The vectorized version is typically 50-100x faster.

### Common Vectorization Patterns

**Replacing conditional loops with boolean indexing:**

In [ ]:
# Slow: loop with conditional
def clip_negative_loop(arr):
    result = arr.copy()
    for i in range(len(arr)):
        if result[i] < 0:
            result[i] = 0
    return result


# Fast: boolean indexing
def clip_negative_vectorized(arr):
    result = arr.copy()
    result[result < 0] = 0
    return result


# Even faster: use built-in
def clip_negative_builtin(arr):
    return np.maximum(arr, 0)

**Computing pairwise operations:**

In [25]:
# Slow: nested loops
def pairwise_diff_loop(x):
    n = len(x)
    result = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            result[i, j] = x[i] - x[j]
    return result


# Fast: broadcasting
def pairwise_diff_broadcast(x):
    return x[:, np.newaxis] - x[np.newaxis, :]


x = np.random.randn(1000)
%timeit pairwise_diff_loop(x)
%timeit pairwise_diff_broadcast(x)

141 ms ± 803 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
699 µs ± 4.88 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


**Applying functions to array elements:**

In [ ]:
# Slow: loop
def transform_loop(arr):
    result = np.empty_like(arr)
    for i in range(len(arr)):
        result[i] = np.exp(-arr[i]**2)
    return result


# Fast: vectorized
def transform_vectorized(arr):
    return np.exp(-arr**2)

### Question

Consider the following function that computes the softmax transformation:

In [ ]:
def softmax_loop(x):
    """Compute softmax using loops."""
    n = len(x)
    exp_x = np.empty(n)
    for i in range(n):
        exp_x[i] = np.exp(x[i])
    total = 0
    for i in range(n):
        total += exp_x[i]
    result = np.empty(n)
    for i in range(n):
        result[i] = exp_x[i] / total
    return result

How would you rewrite this using NumPy vectorization? What is the single-line vectorized version?

### Answer

The vectorized version replaces all three loops with array operations:

In [ ]:
def softmax_vectorized(x):
    """Compute softmax using vectorization."""
    exp_x = np.exp(x)
    return exp_x / np.sum(exp_x)

Or as a single expression: `np.exp(x) / np.sum(np.exp(x))`. For numerical stability, subtract the maximum first: `exp_x = np.exp(x - np.max(x))`.

### Memory Layout and Cache Efficiency

NumPy arrays can be stored in row-major (C) or column-major (Fortran) order. Accessing elements in the storage order is faster because it uses CPU cache efficiently.

In [9]:
# Create arrays with different memory layouts
c_array = np.zeros((1000, 1000), order='C')  # Row-major
f_array = np.zeros((1000, 1000), order='F')  # Column-major

# Row-wise sum is faster for C-order arrays
%timeit c_array.sum(axis=1)
%timeit f_array.sum(axis=1)

# Column-wise sum is faster for F-order arrays
%timeit c_array.sum(axis=0)
%timeit f_array.sum(axis=0)

126 µs ± 858 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
137 µs ± 1.28 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
139 µs ± 434 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
132 µs ± 2.35 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


When performing operations that iterate over one axis, ensure the array layout matches your access pattern.

### Limitations of Vectorization

Vectorization works well when:
- Operations are element-wise or have regular patterns
- NumPy provides the needed functions
- Memory is not a constraint (intermediate arrays fit in RAM)

Vectorization is difficult when:
- Operations depend on previous results (sequential dependencies)
- Logic is complex with many conditionals
- Custom algorithms don't map to NumPy operations

For sequential dependencies and complex logic, we can use JIT compilation (covered later). But first, let's look at parallelization for independent tasks.

## Parallelization with multiprocessing

When your computation can be split into independent tasks, parallelization distributes work across multiple CPU cores. Python's `multiprocessing` module provides a way to do this without requiring special compilers or syntax.

### Why multiprocessing?

Python's Global Interpreter Lock (GIL) prevents multiple threads from executing Python bytecode simultaneously. This means the `threading` module doesn't provide true parallelism for CPU-bound tasks. The `multiprocessing` module sidesteps this by spawning separate Python processes, each with its own interpreter and memory space.

### Using Pool for Parallel Map

The most common pattern is applying a function to many inputs in parallel:

In [ ]:
import multiprocessing as mp
import numpy as np


def compute_bootstrap_mean(args):
    """Compute mean of a bootstrap sample."""
    data, seed = args
    rng = np.random.RandomState(seed)
    sample = rng.choice(data, size=len(data), replace=True)
    return np.mean(sample)


# Generate some data
data = np.random.randn(10000)
n_bootstrap = 1000

# Serial version
def bootstrap_serial(data, n_bootstrap):
    results = []
    for i in range(n_bootstrap):
        results.append(compute_bootstrap_mean((data, i)))
    return results

# Parallel version
def bootstrap_parallel(data, n_bootstrap, n_workers=4):
    args = [(data, i) for i in range(n_bootstrap)]
    with mp.Pool(n_workers) as pool:
        results = pool.map(compute_bootstrap_mean, args)
    return results

```
%timeit bootstrap_serial(data, n_bootstrap)
# 458 ms ± 12.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

%timeit bootstrap_parallel(data, n_bootstrap, n_workers=4)
# 187 ms ± 8.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
```

With 4 workers, the parallel version is about 2-3x faster. The speedup is less than 4x due to process creation overhead and data serialization between processes.

Key points about `Pool.map`:
- Creates a pool of worker processes
- Distributes the input list across workers
- Each worker applies the function to its assigned inputs
- Results are collected and returned in order

The function passed to `pool.map` must be defined at the module level (not inside another function) and must be picklable.

### When to Use Parallelization

Parallelization is effective when:
- Tasks are **independent** (no shared state or dependencies between iterations)
- Each task takes **significant time** (overhead of spawning processes is non-trivial)
- You have **multiple CPU cores** available

Parallelization is less effective when:
- Tasks are very fast (process overhead dominates)
- Tasks share state or need to communicate frequently
- Memory is limited (each process has its own copy of data)

### Question

Consider a Monte Carlo simulation where each iteration:
1. Generates random samples
2. Computes a statistic
3. Returns the result

You want to run 10,000 iterations. Which parallelization approach would you use, and what potential issue should you watch out for?

### Answer

Use `Pool.map` or `ProcessPoolExecutor.map` since each iteration is independent. The key issue is **random number generation**: each worker process needs its own random seed, otherwise they may generate identical random sequences. Pass a unique seed to each task (as shown in the bootstrap example above) or use `np.random.default_rng()` with different seeds per worker.

## Just-in-Time Compilation with Numba

Numba is a JIT compiler that translates Python functions to optimized machine code at runtime. You can keep writing Python loops and get near-C performance.

### Installing Numba

In [13]:
!pip install numba

### The @jit Decorator

The simplest way to use Numba is the `@jit` decorator:

In [ ]:
from numba import jit
import numpy as np


@jit
def sum_array(arr):
    """Sum array elements using a loop."""
    total = 0.0
    for i in range(len(arr)):
        total += arr[i]
    return total


arr = np.random.randn(1000000)

# First call includes compilation time
result = sum_array(arr)

# Subsequent calls are fast
%timeit sum_array(arr)
%timeit np.sum(arr)  # Compare with NumPy

The first call to a JIT-compiled function is slow because Numba compiles the function. Subsequent calls use the compiled version and are fast.

### nopython Mode

By default, Numba tries to compile in "nopython" mode, where no Python objects are used. If it can't, it falls back to "object" mode, which is slower. Always use `nopython=True` (or the shorthand `@njit`) to ensure fast compilation:

In [ ]:
from numba import njit


@njit
def compute_mean(arr):
    """Compute mean with guaranteed fast compilation."""
    total = 0.0
    n = len(arr)
    for i in range(n):
        total += arr[i]
    return total / n

If Numba can't compile in nopython mode, it will raise an error instead of silently falling back to slow code.

### Supported Features

Numba supports a subset of Python and NumPy:

**Supported:**
- Numeric types (int, float, complex)
- NumPy arrays and many NumPy functions
- Loops, conditionals, basic control flow
- Tuples (with fixed types)
- Simple functions

**Not supported:**
- Lists with varying types
- Dictionaries (limited support)
- String operations
- Classes (limited support)
- Many Python built-ins

When in doubt, check the [Numba documentation](https://numba.readthedocs.io/en/stable/reference/pysupported.html).

### Example: Monte Carlo Pi Estimation

Here's a practical example comparing pure Python, NumPy, and Numba:

In [ ]:
import numpy as np
from numba import njit
import random


def estimate_pi_python(n):
    """Estimate pi using Monte Carlo - pure Python."""
    inside = 0
    for _ in range(n):
        x = random.random()
        y = random.random()
        if x*x + y*y <= 1:
            inside += 1
    return 4 * inside / n


def estimate_pi_numpy(n):
    """Estimate pi using Monte Carlo - NumPy."""
    x = np.random.random(n)
    y = np.random.random(n)
    inside = np.sum(x*x + y*y <= 1)
    return 4 * inside / n


@njit
def estimate_pi_numba(n):
    """Estimate pi using Monte Carlo - Numba."""
    inside = 0
    for _ in range(n):
        x = np.random.random()
        y = np.random.random()
        if x*x + y*y <= 1:
            inside += 1
    return 4 * inside / n


n = 10000000

# Warm up Numba
estimate_pi_numba(1000)

%timeit estimate_pi_python(n)
%timeit estimate_pi_numpy(n)
%timeit estimate_pi_numba(n)

Typical results show Numba matching or exceeding NumPy performance while keeping the simple loop structure.

### Question

The following Numba function fails to compile in nopython mode. Why, and how would you fix it?

In [ ]:
@njit
def filter_positive(arr):
    """Return only positive elements."""
    result = []
    for x in arr:
        if x > 0:
            result.append(x)
    return np.array(result)

### Answer

The function fails because Python lists with `append()` are not fully supported in nopython mode. Numba can't know the final size of the list at compile time.

Two fixes:

**Fix 1: Two-pass approach**

In [ ]:
@njit
def filter_positive(arr):
    # First pass: count positives
    count = 0
    for x in arr:
        if x > 0:
            count += 1
    # Second pass: fill result
    result = np.empty(count)
    j = 0
    for x in arr:
        if x > 0:
            result[j] = x
            j += 1
    return result

**Fix 2: Use NumPy boolean indexing outside Numba**

In [ ]:
def filter_positive(arr):
    return arr[arr > 0]  # NumPy is already fast for this

### Parallel Loops with prange

Numba can automatically parallelize loops using `prange`:

In [ ]:
from numba import njit, prange


@njit(parallel=True)
def parallel_sum_squares(arr):
    """Sum of squares using parallel loop."""
    total = 0.0
    for i in prange(len(arr)):
        total += arr[i] ** 2
    return total


arr = np.random.randn(10000000)

# Compare serial vs parallel
@njit
def serial_sum_squares(arr):
    total = 0.0
    for i in range(len(arr)):
        total += arr[i] ** 2
    return total


# Warm up
serial_sum_squares(arr)
parallel_sum_squares(arr)

%timeit serial_sum_squares(arr)
%timeit parallel_sum_squares(arr)

`prange` works like `range` but distributes iterations across CPU cores. Use it when iterations are independent.

### When to Use Numba

Numba is most effective when:
- You have loops that can't be easily vectorized
- Operations involve numerical computations
- The function is called many times

Numba is less effective when:
- The function is called only once (compilation overhead dominates)
- Operations involve strings, complex objects, or I/O
- NumPy already provides an efficient solution

## Interfacing with C/C++ Using Cython

Cython is a language that combines Python syntax with C data types. It compiles to C code that can be built as a Python extension module. Cython offers:

- Gradual optimization: start with Python, add types incrementally
- Direct access to C libraries
- Fine-grained control over memory and performance

### Installing Cython

In [ ]:
!pip install cython

### Basic Cython Workflow

Cython code is written in `.pyx` files and compiled to C extensions. Here's a minimal example:

**example.pyx:**

In [ ]:
%%cython
def sum_python(arr):
    """Pure Python - Cython compiles it."""
    total = 0.0
    for i in range(len(arr)):
        total += arr[i]
    return total

**setup.py:**

In [ ]:
from setuptools import setup
from Cython.Build import cythonize
import numpy as np

setup(
    ext_modules=cythonize("example.pyx"),
    include_dirs=[np.get_include()]
)

Build with: `python setup.py build_ext --inplace`

Then import and use: `from example import sum_python`

### Adding Type Declarations

The power of Cython comes from adding C type declarations. Compare these versions:

In [ ]:
%%cython
# Version 1: Pure Python (slow)
def sum_python(arr):
    total = 0.0
    for i in range(len(arr)):
        total += arr[i]
    return total


# Version 2: Typed Cython (fast)
import numpy as np
cimport numpy as np

def sum_typed(np.ndarray[np.float64_t, ndim=1] arr):
    cdef int i
    cdef int n = arr.shape[0]
    cdef double total = 0.0
    for i in range(n):
        total += arr[i]
    return total

Key type declarations:
- `cdef int i`: C integer for loop variable
- `cdef double total`: C double for accumulator
- `np.ndarray[np.float64_t, ndim=1]`: typed NumPy array

These declarations eliminate Python overhead and enable C-level operations.

### Disabling Bounds Checking

By default, Cython checks array bounds on every access. For performance-critical code, you can disable this:

In [ ]:
%%cython
import numpy as np
cimport numpy as np
cimport cython


@cython.boundscheck(False)
@cython.wraparound(False)
def sum_fast(np.ndarray[np.float64_t, ndim=1] arr):
    cdef int i
    cdef int n = arr.shape[0]
    cdef double total = 0.0
    for i in range(n):
        total += arr[i]
    return total

- `@boundscheck(False)`: skip array bounds checking
- `@wraparound(False)`: skip negative index handling

Only disable these when you're certain your indices are valid.

### Question

Consider this Cython function for computing pairwise Euclidean distances:

In [ ]:
%%cython
import numpy as np
cimport numpy as np

def pairwise_distances(np.ndarray[np.float64_t, ndim=2] X):
    cdef int n = X.shape[0]
    cdef int d = X.shape[1]
    cdef np.ndarray[np.float64_t, ndim=2] result = np.zeros((n, n))

    for i in range(n):
        for j in range(n):
            dist = 0.0
            for k in range(d):
                diff = X[i, k] - X[j, k]
                dist += diff * diff
            result[i, j] = dist ** 0.5
    return result

What variable declarations are missing that would make this code faster?

### Answer

The loop variables `i`, `j`, `k` and the intermediate variables `dist` and `diff` need `cdef` declarations:

In [ ]:
%%cython
def pairwise_distances(np.ndarray[np.float64_t, ndim=2] X):
    cdef int n = X.shape[0]
    cdef int d = X.shape[1]
    cdef int i, j, k
    cdef double dist, diff
    cdef np.ndarray[np.float64_t, ndim=2] result = np.zeros((n, n))

    for i in range(n):
        for j in range(n):
            dist = 0.0
            for k in range(d):
                diff = X[i, k] - X[j, k]
                dist += diff * diff
            result[i, j] = dist ** 0.5
    return result

Without `cdef`, each variable is a Python object, requiring type checking and memory allocation on every operation.

### Using Typed Memoryviews

Modern Cython recommends typed memoryviews instead of NumPy array syntax:

In [ ]:
%%cython
import numpy as np
cimport cython


@cython.boundscheck(False)
@cython.wraparound(False)
def sum_memoryview(double[:] arr):
    cdef int i
    cdef int n = arr.shape[0]
    cdef double total = 0.0
    for i in range(n):
        total += arr[i]
    return total

Memoryviews (`double[:]` for 1D, `double[:, :]` for 2D) work with any object that supports the buffer protocol, not just NumPy arrays.

### Calling C Libraries

Cython can directly call C functions. For example, using the C math library:

In [ ]:
%%cython
from libc.math cimport sqrt, exp, log

cdef double fast_gaussian(double x, double mu, double sigma):
    cdef double z = (x - mu) / sigma
    return exp(-0.5 * z * z) / (sigma * sqrt(2 * 3.14159265358979))

This avoids the overhead of Python's `math` module.

### Pure Python Mode

Cython 3.0+ supports pure Python syntax using type annotations:

In [ ]:
# example_pure.py
import cython
import numpy as np


@cython.cfunc
def compute_sum(arr: cython.double[:]) -> cython.double:
    i: cython.int
    n: cython.int = arr.shape[0]
    total: cython.double = 0.0
    for i in range(n):
        total += arr[i]
    return total

This can be run as regular Python or compiled with Cython for speed.

### Cython vs Numba

| Feature | Numba | Cython |
|---------|-------|--------|
| Ease of use | Simple decorator | Requires compilation step |
| Compilation | JIT (at runtime) | AOT (ahead of time) |
| C library access | Limited | Full access |
| Debugging | Harder | Easier (generates readable C) |
| Distribution | Include Numba dependency | Distribute compiled extensions |
| Learning curve | Lower | Higher |

**Use Numba when:**
- You want quick speedups with minimal code changes
- Your algorithm uses standard numerical operations
- You don't need to call external C libraries

**Use Cython when:**
- You need to wrap existing C/C++ code
- You want fine-grained control over optimization
- You're distributing a package and want to avoid runtime dependencies

## Putting It All Together

Here's a workflow for optimizing Python code:

1. **Write correct code first.** Don't optimize prematurely. Make sure your code produces correct results.

2. **Profile to find bottlenecks.** Use `cProfile` to identify which functions take the most time.

3. **Try vectorization.** Can the slow code be rewritten using NumPy operations? This is often the simplest solution.

4. **Consider Numba.** If vectorization doesn't work, try adding `@njit`. This works well for numerical loops.

5. **Use Cython for special cases.** When you need to call C libraries or want maximum control, Cython is the tool.

6. **Verify correctness.** After optimization, ensure the code still produces correct results. Compare outputs against the original implementation.

### Example: Optimizing a Statistical Function

Let's optimize a function that computes the log-likelihood of a Gaussian mixture model:

In [ ]:
import numpy as np


def gmm_log_likelihood_python(X, weights, means, covs):
    """Compute GMM log-likelihood - pure Python."""
    n_samples = X.shape[0]
    n_components = len(weights)

    log_likelihood = 0.0
    for i in range(n_samples):
        sample_prob = 0.0
        for k in range(n_components):
            diff = X[i] - means[k]
            cov_inv = np.linalg.inv(covs[k])
            det = np.linalg.det(covs[k])
            exponent = -0.5 * diff @ cov_inv @ diff
            prob = weights[k] * np.exp(exponent) / np.sqrt((2*np.pi)**len(diff) * det)
            sample_prob += prob
        log_likelihood += np.log(sample_prob)

    return log_likelihood

**Step 1: Profile**

The nested loops and repeated matrix inversions are clear bottlenecks.

**Step 2: Vectorize**

In [ ]:
def gmm_log_likelihood_vectorized(X, weights, means, covs):
    """Compute GMM log-likelihood - vectorized."""
    n_samples, n_features = X.shape
    n_components = len(weights)

    # Precompute inverse and determinant for each component
    precisions = [np.linalg.inv(cov) for cov in covs]
    log_dets = [np.log(np.linalg.det(cov)) for cov in covs]

    # Compute log probabilities for all samples and components
    log_probs = np.zeros((n_samples, n_components))
    for k in range(n_components):
        diff = X - means[k]  # (n_samples, n_features)
        mahal = np.sum(diff @ precisions[k] * diff, axis=1)
        log_probs[:, k] = (np.log(weights[k])
                          - 0.5 * n_features * np.log(2*np.pi)
                          - 0.5 * log_dets[k]
                          - 0.5 * mahal)

    # Log-sum-exp for numerical stability
    max_log_probs = np.max(log_probs, axis=1)
    log_likelihood = np.sum(max_log_probs +
                           np.log(np.sum(np.exp(log_probs - max_log_probs[:, np.newaxis]), axis=1)))

    return log_likelihood

This version moves the sample loop into NumPy operations, keeping only the component loop in Python.

### Question

Looking at the vectorized GMM function above, what additional optimization could Numba provide, and what limitations might you encounter?

### Answer

Numba could potentially speed up the remaining component loop, but there are limitations:

1. **`np.linalg.inv` and `np.linalg.det`** are supported in Numba, but performance gains may be modest since they're already calling optimized LAPACK routines.

2. **The main benefit** would come from JIT-compiling the Mahalanobis distance computation if we unrolled it into explicit loops.

3. **Limitation:** For this problem, the vectorized NumPy version is likely already near-optimal because the heavy lifting (matrix operations) is done by optimized BLAS/LAPACK libraries. Numba shines more when you have complex control flow that can't be vectorized.

The best approach here is probably the vectorized version, possibly using `scipy.stats.multivariate_normal.logpdf` which is already optimized.

## Summary

* **Profile first:** Use `timeit` and `cProfile` to identify bottlenecks before optimizing.

* **Vectorize with NumPy:** Replace Python loops with array operations. This is often the simplest and most effective optimization.

* **Use Numba for numerical loops:** Add `@njit` to functions with loops that can't be vectorized. Numba compiles Python to machine code with minimal code changes.

* **Use Cython for C integration:** When you need to wrap C libraries or want maximum control, Cython compiles Python-like code to C extensions.

* **Parallelize independent tasks:** Use `multiprocessing.Pool` or `ProcessPoolExecutor` to distribute work across CPU cores when tasks are independent.

* **Verify correctness:** Always test that optimized code produces the same results as the original.

## Recommended Resources

* [Python Profiling Guide](https://realpython.com/python-profiling/)
* [NumPy Performance Tips](https://numpy.org/doc/stable/user/basics.indexing.html)
* [Numba Documentation](https://numba.readthedocs.io/en/stable/)
* [Cython Documentation](https://cython.readthedocs.io/en/latest/)
* [High Performance Python (Book)](https://www.oreilly.com/library/view/high-performance-python/9781492055013/)